# 데이터 크롤링 실습

트젠퀴 트럭 부스의 셋 리스트를 엑셀로 입력 받아서, 해당 음악에 대한 길이를 지니에서 beatifulsoup과 selenium으로 데이터를 찾아서 다시 엑셀 형태로 출력하자

0 엑셀로 데이터 받기

-입력 폼은 번호/아티스트/제목

1 지니에서 음악 검색

2 첫번째로 나오는 것 클릭해서 들어가서, 아티스트 / 제목 / 재생시간 따와서 append 하기 - 검색해서 나오는게 없으면 에러처리하기

-출력 폼은 번호/(입력)아티스트/(입력)제목/(검색결과)아티스트/(검색결과)제목/(검색결과)재생시간/비고

참고 : https://mrxx.tistory.com/69?category=760049, 언더스코어 강의, nims academy 강의

In [1]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import timedelta
import datetime
import urllib.parse
import numpy as np
import pandas as pd
from time import sleep
import pickle

import os
import glob

In [2]:
os.getcwd()

'C:\\Users\\TAK\\Desktop\\Data\\Lectures\\Others\\기타 코딩'

In [3]:
glob.glob("*.*")

['190503 데이터크롤링 실습.ipynb', 'input_test.xlsx']

In [4]:
#잘 불러와지나 테스트

df0=pd.read_excel('input_test.xlsx',header = 0,encoding='UTF-8').fillna(0) #header : 앞에 1줄 제외하려고. fillna(0)은 빈칸을 0으로 만들기 위해서
#resd_excel에서 csv파일을 위와 같은 형태의 식으로 부르면 오류가 생김. csv 파일에서는 header 기능을 쓸 수 없을 것으로 보이는 데 그 때문 같다.

df0.head()

,Num,Input_artist,Input_Name
0,1,레드 벨벳,빨간 맛
1,2,레드 벨벳,ice cream cake
2,3,기하서,복소해석학


In [5]:
n=len(df0.index)

In [6]:
from selenium import webdriver #리모컨 가져온다는 느낌

In [7]:
#test
i=0
inArtist = df0.iloc[i][1]
inSongName = df0.iloc[i][2]

print (inArtist,inSongName)

레드 벨벳 빨간 맛


In [8]:
type(inArtist)

str

In [9]:
# 제어용 웹 드라이버 설정
browser=webdriver.Chrome('C:/chromedriver/chromedriver.exe')

In [10]:
#한글을 url에 쓰이는 utf-8 코드로 변환해보기_1

AAAAA=inArtist+" "+inSongName
print (AAAAA[0])
tt=str("레".encode("utf-8"))
YY=tt.split("\\x")
PP=""
for i in range(len(YY)) :
    if i == len(YY)-1 :
        PP=PP+"%"+YY[i][:-1]    
    elif i != 0 :
        PP=PP+"%"+YY[i]

레


In [11]:
#한글을 url에 쓰이는 utf-8 코드로 변환해보기_2
ppp=""
for _ in range(len(AAAAA)) :
    if AAAAA[_]==" " :
        ppp=ppp+"%20"
    else :
        tt=str(AAAAA[_].encode("utf-8"))
        YY=tt.split("\\x")
        for i in range(len(YY)) :
            if i == len(YY)-1 :
                ppp=ppp+"%"+YY[i][:-1]
            elif i != 0 :
                ppp=ppp+"%"+YY[i]

print (ppp)

tttttt="%EB%A0%88%EB%93%9C%20%EB%B2%A8%EB%B2%B3%20%EB%B9%A8%EA%B0%84%20%EB%A7%9B"

ppp.upper()==tttttt

%eb%a0%88%eb%93%9c%20%eb%b2%a8%eb%b2%b3%20%eb%b9%a8%ea%b0%84%20%eb%a7%9b


True

In [12]:
url="https://www.genie.co.kr/search/searchMain?query="+inArtist+" "+inSongName
url

'https://www.genie.co.kr/search/searchMain?query=레드 벨벳 빨간 맛'

In [13]:
browser.get(url)

In [14]:
browser.current_url #이게 그냥 ord 함수로 ascii로 바꾼 걸 16진수로 바꾼건 아니고 utf-8코드로 바꿔야 함.

'https://www.genie.co.kr/search/searchMain?query=%EB%A0%88%EB%93%9C%20%EB%B2%A8%EB%B2%B3%20%EB%B9%A8%EA%B0%84%20%EB%A7%9B'

In [15]:
url=browser.current_url #크롬 드라이버에서 현재 접속해 이쓴ㄴ 주소

In [16]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
resp = requests.get(url, headers = headers)
soup = BeautifulSoup(resp.text, 'html.parser')

# 헤더를 지정하지 않고 크롤링을 하면 보안 문제로 접속이 안됨. 아래 블로그에서 유저 정보 참고해서 header에 값 대입해서 하면 됨
# http://blog.naver.com/PostView.nhn?blogId=kiddwannabe&logNo=221185808375

In [17]:
soup.find_all('tr', attrs={'class':'list'})
#tr 들어간거 다 찾는건데 개중에 class가 list인 것만

[<tr class="list" songid="87270282">
 <td class="check"><input class="select-check" title="빨간 맛 (red flavor)
 " type="checkbox"/></td>
 <td class="number">1</td>
 <td><a class="cover" href="#" onclick="fnViewAlbumLayer('80964658');return false;"><span class="mask"></span><img alt="빨간 맛 (Red Flavor)" onerror="this.src='//image.genie.co.kr/imageg/web/common/blank_68.gif';" src="//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/080/964/658/80964658_1499421855077_1_140x140.JPG"/></a></td>
 <td class="link"><a class="btn-basic btn-info" href="#" onclick="fnViewSongInfo('87270282');return false;">곡 제목 정보 페이지</a></td>
 <td class="info">
 <a class="title ellipsis" href="#" onclick="fnPlaySong('87270282;','1');return false;" title="빨간 맛 (red flavor)
 ">
 <span class="icon icon-title">TITLE</span>
 <span class="t_point">빨간</span> <span class="t_point">맛</span> (Red Flavor) </a>
 <a class="artist ellipsis" href="#" onclick="fnViewArtist('80284018');return false;">Red Velvet (<span class="t_point">레드벨벳</span>

In [18]:
AA=str(soup.find_all('tr', attrs={'class':'list'})[0]) #현재 노래 별로 block이 잡혀있어서, 이렇게 하면 제일 먼저 검색된 게 올라옴

In [19]:
#노래 ID 추출하기 - 지니의 경우 노래 id별로 url이 되어있어서 

BB=AA.split('<')

CC=BB[1]

In [20]:
CC

'tr class="list" songid="87270282">\n'

In [21]:
CC.find('songid') #여기서 find하기 전에 search로 에러처리 해줘야

16

In [22]:
CC[23]

'"'

In [23]:
?CC.find

Docstring:
S.find(sub[, start[, end]]) -> int

Return the lowest index in S where substring sub is found,
such that sub is contained within S[start:end].  Optional
arguments start and end are interpreted as in slice notation.

Return -1 on failure.
Type:      builtin_function_or_method


In [24]:
CC.find('"',24)

32

In [25]:
SongID=CC[24:32] #근데 이렇게 안하고 그냥 숫자만 추출하기로 해도 되려나 싶음.

In [26]:
url = "https://www.genie.co.kr/detail/songInfo?xgnm="+SongID

In [27]:
browser.get(url)

In [28]:
#songid로 지정된 페이지에서 파싱

resp = requests.get(url, headers = headers)
soup1 = BeautifulSoup(resp.text, 'html.parser')


In [29]:
#곡 이름 추출
soup1.find_all('h2', attrs={'class':'name'})

[<h2 class="name">
 					
 					빨간 맛 (Red Flavor)
 				</h2>]

In [30]:
FF=str(soup1.find_all('h2', attrs={'class':'name'})[0].text)
FF

'\n\t\t\t\t\t\n\t\t\t\t\t빨간 맛 (Red Flavor)\n\t\t\t\t'

In [31]:
print(FF)


					
					빨간 맛 (Red Flavor)
				


In [32]:
kkk=FF.find("\t")
print(kkk)
print(FF)

1

					
					빨간 맛 (Red Flavor)
				


In [33]:
#공백 없애기
a=[]
i=kkk
done = False
while done == False :
    if i >= len(FF) :
        done = True
    elif FF[i:i+1]=="\t" :
        i+=2
    else :
        a.append(i)
        i+=1
b=""
for i in range(len(a)) :
    b=b+FF[a[i]-1]
  
print(b)

빨간 맛 (Red Flavor)


In [34]:
#곡 관련 세부 정보 : 가수, 앨범, 장르, 재생시간, +a 순
soup1.find_all('span', attrs={'class':'value'})

[<span class="value"><a href="#" onclick="fnGoMore('artistInfo','80284018');return false;">Red Velvet (레드벨벳)</a></span>,
 <span class="value"><a href="#" onclick="fnGoMore('albumInfo','80964658');return false;">The Red Summer - Summer Mini Album</a></span>,
 <span class="value">가요 / 댄스</span>,
 <span class="value">03:11</span>,
 <span class="value"><a href="#" onclick="fnViewArtist('80292177'); return false;">KENZIE</a></span>,
 <span class="value"><a href="#" onclick="fnViewArtist('80590951'); return false;">Daniel Caesar</a>,<a href="#" onclick="fnViewArtist('80590952'); return false;">Ludwig Lindell</a></span>,
 <span class="value"><a href="#" onclick="fnViewArtist('80590954'); return false;">Caesar &amp; Loui</a></span>]

In [35]:
soup1.find_all('span', attrs={'class':'value'})[0]

<span class="value"><a href="#" onclick="fnGoMore('artistInfo','80284018');return false;">Red Velvet (레드벨벳)</a></span>

In [36]:
soup1.find_all('span', attrs={'class':'value'})[3]

<span class="value">03:11</span>

In [37]:
DD=str(soup1.find_all('span', attrs={'class':'value'})[0])
EE=str(soup1.find_all('span', attrs={'class':'value'})[3])

t=EE.find(":")
timetime=EE[t-2:t+3]

In [38]:
GG=DD.split(">")[-3]
GG

'Red Velvet (레드벨벳)</a'

In [39]:
aa=DD.split(">")[-3].find("<")
artistname=DD.split(">")[-3][:aa]

In [1]:
print(timetime,artistname)

NameError: name 'timetime' is not defined

In [2]:
a="d"
b="c"
d=[a,b]
print(d)

['d', 'c']


테스트 결과 :

1 user info를 입력하지 않았더니 보안 문제로 걸렸음
2 셀레니움으로 클릭을 해야하나 했더니, 운이 좋게도 url 구조가 쉬워서 클릭 없이도 파싱할 수 있게 되었음
3 